# Technology preview: Femtocode

Although it was the focus of today's tutorials, Histogrammar is not an end in itself. It's part of a larger system in development to replace private skims with a query system.

The idea is that all the big data resides on a server, like MiniAOD in EOS, and instead of downloading skims for interactive analysis, you (the physicists) perform analyses directly on the central server by submitting queries and getting results as aggregations.

Of course, it has to be considerably faster than current Grid jobs. You have to get each plot back in seconds (or less) to be productive. Some early work shows that this is possible: [see talk](https://indico.cern.ch/event/630641/)).

The query system wouldn't accept C++ strings, as in our examples today, but a new query language called Femtocode. The "femto" refers to the fact that it's intended for tiny snippets. This query language is both

   * more high-level than C++, having a Pythonic syntax apart from simplified function declarations (you don't have to type "`lambda`")
   * and faster than the equivalent C++, because it has additional constraints. It's only used for querying, after all, not application development.

The example below shows what this would look like. The cell doesn't function (don't try to evaluate it), but the plot shown below it was generated from real Femtocode. (4.5x faster than objects-on-heap C++, should be 20x.)

In [ ]:
from femtocode.remote import RemoteSession

session = RemoteSession("http://doesnt.exist.yet.fnal.gov")

workflow = session.source("ZZ_13TeV_pythia8")            # pull from a named dataset
       .define(goodmuons = "muons.filter($1.pt > 5)")    # muons with pt > 5 are good
       .filter("goodmuons.size >= 2")                    # keep events with at least two
       .define(dimuon = """
           mu1, mu2 = goodmuons.maxby($1.pt, 2);         # pick the top two by pt
           energy = mu1.E + mu2.E;                       # compute imploded energy/momentum
           px = mu1.px + mu2.px;
           py = mu1.py + mu2.py;
           pz = mu1.pz + mu2.pz;

           rec(mass = sqrt(energy**2 - px**2 - py**2 - pz**2),
               pt = sqrt(px**2 + py**2),
               phi = atan2(py, px),
               eta = log((energy + pz)/(energy - pz))/2) # construct a record as output
           """)
       .bundle(                                          # make a bundle of plots (Histogrammar 2.0)
           mass = bin(120, 60, 120, "dimuon.mass"),      # using the variables we’ve made
           pt = bin(100, 0, 100, "dimuon.pt"),
           eta = bin(100, -5, 5, "dimuon.eta"),
           phi = bin(314, 0, 2*pi, "dimuon.phi + pi"),
           muons = loop("goodmuons", "mu", bundle(       # also make plots with one muon per entry
               pt = bin(100, 0, 100, "mu.pt"),
               eta = bin(100, -5, 5, "mu.eta"),
               phi = bin(314, -pi, pi, "mu.phi")
           ))
       )

pending = workflow.submit()                              # submit the query
pending["mass"].plot()                                   # and plot results while they accumulate
pending["muons"]["pt"].plot()                            # (they’ll be animated)

blocking = pending.await()                               # stop the code until the result is in

massplot = blocking.plot.root("dimuon mass [GeV]")       # convert to a familiar format, like ROOT
massplot.Draw()                                          # and use that package’s tools

![](c1.png)

For more, see the README on the [Femtocode GitHub page](https://github.com/diana-hep/femtocode).